Dado que el entrenamiento de redes neuronales es una tarea  muy costosa, **se recomienda ejecutar el notebooks en [Google Colab](https://colab.research.google.com)**, por supuesto también se puede ejecutar en local.

Al entrar en [Google Colab](https://colab.research.google.com) bastará con hacer click en `upload` y subir este notebook. No olvide luego descargarlo en `File->Download .ipynb`

**El examen deberá ser entregado con las celdas ejecutadas, si alguna celda no está ejecutadas no se contará.**

El examen se divide en tres partes, con la puntuación que se indica a continuación. La puntuación máxima será 10.

    
- [Actividad 1: Redes Recurrentes](#actividad_1): 10 pts
    - [Cuestión 1](#3.1): 2.5 pt
    - [Cuestión 2](#3.2): 2.5 pt
    - [Cuestión 3](#3.3): 2.5 pts
    - [Cuestión 4](#3.4): 1.25 pts
    - [Cuestión 5](#3.5): 1.25 pts



In [16]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

<a name='actividad_1'></a>
# Actividad 1: Redes Recurrentes


- [Cuestión 1](#3.1): 2.5 pt
- [Cuestión 2](#3.2): 2.5 pt
- [Cuestión 3](#3.3): 2.5 pts
- [Cuestión 4](#3.4): 1.25 pts
- [Cuestión 5](#3.5): 1.25 pts

Vamos a usar un dataset de las temperaturas mínimas diarias en Melbourne. La tarea será la de predecir la temperatura mínima en dos días. Puedes usar técnicas de series temporales vistas en otras asignaturas, pero no es necesario.


In [17]:
dataset_url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-min-temperatures.csv'
data_dir = tf.keras.utils.get_file('daily-min-temperatures.csv', origin=dataset_url)

In [18]:
df = pd.read_csv(data_dir, parse_dates=['Date'])
df.head()

,Date,Temp
0,1981-01-01,20.7
1,1981-01-02,17.9
2,1981-01-03,18.8
3,1981-01-04,14.6
4,1981-01-05,15.8


In [19]:
temperatures = df['Temp'].values
print('number of samples:', len(temperatures))
train_data = temperatures[:3000]
test_data = temperatures[3000:]
print('number of train samples:', len(train_data))
print('number of test samples:', len(test_data))
print('firsts trainn samples:', train_data[:10])

number of samples: 3650
number of train samples: 3000
number of test samples: 650
firsts trainn samples: [20.7 17.9 18.8 14.6 15.8 15.8 15.8 17.4 21.8 20. ]


<a name='3.1'></a>
## Cuestión 1: Convierta `train_data` y `test_data`  en ventanas de tamaño 5, para predecir el valor en 2 días

En la nomenclatura de [Introduction_to_RNN_Time_Series.ipynb](https://github.com/ezponda/intro_deep_learning/blob/main/class/RNN/Introduction_to_RNN_Time_Series.ipynb)
```python
past, future = (5, 2)
```

Para las primeras 10 muestras de train_data `[20.7, 17.9, 18.8, 14.6, 15.8, 15.8, 15.8, 17.4, 21.8, 20. ]` el resultado debería ser:

```python
x[0] : [20.7, 17.9, 18.8, 14.6, 15.8] , y[0]: 15.8
x[1] : [17.9, 18.8, 14.6, 15.8, 15.8] , y[1]: 17.4
x[2] : [18.8, 14.6, 15.8, 15.8, 15.8] , y[2]: 21.8
x[3] : [14.6, 15.8, 15.8, 15.8, 17.4] , y[3]: 20.             
```

In [20]:
# windowing function
def create_windows_np(data, window_size, horizon, shuffle=False):
    """
    Creates a dataset from the given time series data using NumPy.

    Parameters:
    data (np.ndarray): Time series data with one dimension.
    window_size (int): The number of past time steps to use as input features.
    horizon (int): The number of future time steps to predict.
    shuffle (bool): Shuffle the windows or not.

    Returns:
    tuple: A tuple containing the input-output pairs (windows, targets) as NumPy arrays.
    """

    X, y = [], []
    for i in range(len(data) - window_size - horizon + 1):
        X.append(data[i:i+window_size])
        y.append(data[i+window_size+horizon-1])

    X, y = np.array(X, dtype=np.float32)[..., np.newaxis], np.array(y, dtype=np.float32)

    if shuffle:
        indices = np.arange(len(X))
        np.random.shuffle(indices)
        X, y = X[indices], y[indices]

    return X, y

In [21]:
past, future = (5, 2)
##X_train, y_train = ...
X_train, y_train = create_windows_np(train_data, past, future, shuffle=False)
X_test, y_test = create_windows_np(test_data, past, future, shuffle=False)
print(f'Train shape: {X_train.shape}')
print(f'Test shape: {X_test.shape}')

for i in range(4):
    x_list = [round(float(v), 1) for v in X_train[i, :, 0]]
    y_val = round(float(y_train[i]), 1)
    print(f"x[{i}] : {x_list} , y[{i}]: {y_val}")

Train shape: (2994, 5, 1)
Test shape: (644, 5, 1)
x[0] : [20.7, 17.9, 18.8, 14.6, 15.8] , y[0]: 15.8
x[1] : [17.9, 18.8, 14.6, 15.8, 15.8] , y[1]: 17.4
x[2] : [18.8, 14.6, 15.8, 15.8, 15.8] , y[2]: 21.8
x[3] : [14.6, 15.8, 15.8, 15.8, 17.4] , y[3]: 20.0


<a name='3.2'></a>
## Cuestión 2: Cree un modelo recurrente de dos capas GRU para predecir con las ventanas de la cuestión anterior.


In [22]:
#inputs = keras.layers.Input(shape=(..., ...))
inputs = keras.layers.Input(shape=(past, 1), name='input')
GRU_1 = layers.GRU(64, return_sequences=True, name='GRU_1')(inputs)
GRU_2 = layers.GRU(64, return_sequences=False, name='GRU_2')(GRU_1)
outputs = layers.Dense(1, name='output')(GRU_2)

model = keras.Model(inputs=inputs, outputs=outputs, name='rnn_example')
model.compile(
    optimizer='adam',
    loss='mean_squared_error',  # Changed loss function to mean_squared_error for regression
    metrics=['mean_absolute_error']  # Changed metric to mean_absolute_error
)

#...
#model = keras.Model(inputs=..., outputs=...)
#model.compile(...)
model.summary()

Model: "rnn_example"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input (InputLayer)              │ (None, 5, 1)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ GRU_1 (GRU)                     │ (None, 5, 64)          │        12,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ GRU_2 (GRU)                     │ (None, 64)             │        24,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 37,889 (148.00 KB)

 Trainable params: 37,889 (148.00 KB)

 Non-trainable params: 0 (0.00 B)

In [23]:
es_callback = keras.callbacks.EarlyStopping(
    monitor="val_loss", min_delta=0, patience=10)

history = model.fit(
    X_train, y_train,
    epochs=200,
    validation_split=0.2, shuffle=True, batch_size = 64, callbacks=[es_callback]
)

Epoch 1/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 5s 26ms/step - loss: 78.5155 - mean_absolute_error: 7.6134 - val_loss: 20.3153 - val_mean_absolute_error: 3.6056
Epoch 2/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 16.4085 - mean_absolute_error: 3.2437 - val_loss: 16.9220 - val_mean_absolute_error: 3.3210
Epoch 3/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 16.1994 - mean_absolute_error: 3.2295 - val_loss: 16.3777 - val_mean_absolute_error: 3.2300
Epoch 4/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 14.0804 - mean_absolute_error: 2.9449 - val_loss: 12.7179 - val_mean_absolute_error: 2.7776
Epoch 5/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 11.7855 - mean_absolute_error: 2.6756 - val_loss: 10.8725 - val_mean_absolute_error: 2.5484
Epoch 6/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 10.2856 - mean_absolute_error: 2.5040 - val_loss: 10.0214 - val_mean_absolute_error: 2.4432
Epoch 7/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 9.7768 - mean_absolute_error

In [24]:
results = model.evaluate(X_test, y_test, verbose=1)
print('Test Loss: {}'.format(results))

21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 6.8740 - mean_absolute_error: 2.0418
Test Loss: [6.978241920471191, 2.058886766433716]


<a name='3.3'></a>
## Cuestión 3: Añada más features a la series temporal, por ejemplo `portion_year`. Cree un modelo que mejore al anterior.


In [25]:
## Puede añadir más features
df['portion_year'] = df['Date'].dt.dayofyear / 365.0
df_multi = df[['Temp', 'portion_year']].copy()

## train - test split
train_dataM = df_multi.iloc[:3000].copy()
test_dataM = df_multi.loc[3000:, :].copy()

In [26]:
## Create windows
window_size = past
horizon = future

# n features = 2 --> temp, portion_year
n_features = 2

columns = df_multi.columns
target_col_idx = columns.get_loc('Temp')

## se define una nueva función donde sea multivariables

def create_windows_multivariate_np(data, window_size, horizon, target_col_idx, shuffle=False):
    """
    Creates a dataset from the given time series data using NumPy.

    Parameters:
    data (np.ndarray or pd.DataFrame): Time series data with multiple features.
    window_size (int): The number of past time steps to use as input features.
    horizon (int): The number of future time steps to predict.
    target_col_idx (int): The index of the target column in the input data.
    shuffle (bool): Whether to shuffle the data or not.

    Returns:
    tuple: A tuple containing the input-output pairs (X, y) as NumPy arrays.
    """
    if isinstance(data, pd.DataFrame):
        data = data.values

    X, y = [], []
    for i in range(len(data) - window_size - horizon + 1):
        X.append(data[i:i+window_size, :])
        y.append(data[i+window_size+horizon-1, target_col_idx])

    X, y = np.array(X), np.array(y)

    if shuffle:
        indices = np.arange(X.shape[0])
        np.random.shuffle(indices)
        X, y = X[indices], y[indices]

    return X, y
######################################################################


##X_train, y_train = ...
##X_test, y_test = ...

X_trainM, y_trainM = create_windows_multivariate_np(
    train_dataM, window_size, horizon, target_col_idx=target_col_idx, shuffle=True)
X_testM, y_testM = create_windows_multivariate_np(
    test_dataM, window_size, horizon, target_col_idx=target_col_idx, shuffle=False)



In [27]:
##inputs = keras.layers.Input(shape=(..., ...))
##...
inputsM = keras.layers.Input(shape=(window_size, n_features), name='inputM')
GRU_1M = layers.GRU(64, return_sequences=True, name='GRU_1M')(inputsM)
GRU_2M = layers.GRU(64, return_sequences=False, name='GRU_2M')(GRU_1M)
outputsM = layers.Dense(1, name='output')(GRU_2M)

modelM = keras.Model(inputs=inputsM, outputs=outputsM, name='rnn_exampleM')
modelM.compile(
    optimizer='adam',
    loss='mean_squared_error',  # Changed loss function to mean_squared_error for regression
    metrics=['mean_absolute_error']  # Changed metric to mean_absolute_error
)

#...
#model = keras.Model(inputs=..., outputs=...)
#model.compile(...)
modelM.summary()


Model: "rnn_exampleM"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ inputM (InputLayer)             │ (None, 5, 2)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ GRU_1M (GRU)                    │ (None, 5, 64)          │        13,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ GRU_2M (GRU)                    │ (None, 64)             │        24,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 38,081 (148.75 KB)

 Trainable params: 38,081 (148.75 KB)

 Non-trainable params: 0 (0.00 B)

In [28]:
es_callback = keras.callbacks.EarlyStopping(
    monitor="val_loss", min_delta=0, patience=10)

history = modelM.fit(
    X_trainM, y_trainM,
    epochs=200,
    validation_split=0.2, shuffle=True, batch_size = 64, callbacks=[es_callback]
)

Epoch 1/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - loss: 84.3831 - mean_absolute_error: 7.9074 - val_loss: 16.2890 - val_mean_absolute_error: 3.2447
Epoch 2/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 17.1721 - mean_absolute_error: 3.2861 - val_loss: 14.9385 - val_mean_absolute_error: 3.0979
Epoch 3/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 13.9826 - mean_absolute_error: 2.9364 - val_loss: 12.2658 - val_mean_absolute_error: 2.7778
Epoch 4/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 12.2614 - mean_absolute_error: 2.6961 - val_loss: 10.1452 - val_mean_absolute_error: 2.5050
Epoch 5/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 10.2782 - mean_absolute_error: 2.4875 - val_loss: 9.3469 - val_mean_absolute_error: 2.4129
Epoch 6/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 9.6243 - mean_absolute_error: 2.3921 - val_loss: 8.8772 - val_mean_absolute_error: 2.3561
Epoch 7/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 9.2231 - mean_absolute_error: 2

In [29]:
resultsM = modelM.evaluate(X_testM, y_testM, verbose=1)
print('Test Loss: {}'.format(resultsM))

21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 6.8752 - mean_absolute_error: 2.0424
Test Loss: [6.965940952301025, 2.066124439239502]


<a name='3.4'></a>
## Cuestión 4: ¿En cuáles de estas aplicaciones se usaría un arquitectura 'many-to-one'?

**a)** Clasificación de sentimiento en textos

**b)** Verificación de voz para iniciar el ordenador.

**c)** Generación de música.

**d)** Un clasificador que clasifique piezas de música según su autor.


RESPUESTAS A, B Y D


<a name='3.5'></a>
## Cuestión 5: ¿Qué ventajas aporta el uso de word embeddings?

**a)** Permiten reducir la dimensión de entrada respecto al one-hot encoding.

**b)** Permiten descubrir la similaridad entre palabras de manera más intuitiva que con one-hot encoding.

**c)** Son una manera de realizar transfer learning en nlp.

**d)** Permiten visualizar las relaciones entre palabras con métodos de reducción de dimensioones como el PCA.

TODAS LAS RESPUETAS SON CORRECTAS
